In [26]:
using PyPlot
using Profile

In [25]:
using DataStructures

function c2i(cor, shape)
    i, j = cor
    i -= 1
    j -= 1
    n1, n2 = shape
    idx = j * n1 + i
    return idx + 1
end


function i2c(idx, shape)
    n1, n2 = shape
    idx -= 1
    j = div(idx, n1)
    i = idx % n1
    return i + 1, j + 1
end


function my_dijkstra(hweight, vweight, source)
    m, n = size(hweight, 1), size(hweight, 2)+1
    @assert size(vweight) == (m-1, n)
    
    shape = m, n
    result = zeros(m, n) .+ Inf
    done = zeros(Bool, m, n)
    pq = PriorityQueue{Int, Float64}()
    
    i, j = source
    done[i, j] = true
    result[i, j] = 0
    
    while true
        ij_dist = result[i, j]
        
        if (i!=1) && (!done[i-1, j])
            tmp = ij_dist + vweight[i-1, j]
            if tmp < result[i-1, j]
                result[i-1, j] = tmp
                pq[c2i((i-1, j), shape)] = tmp
            end
        end
        if (i!=m) && (!done[i+1, j])
            tmp = ij_dist + vweight[i, j]
            if tmp < result[i+1, j]
                result[i+1, j] = tmp
                pq[c2i((i+1, j), shape)] = tmp
            end
        end
        if (j!=1) && (!done[i, j-1])
            tmp = ij_dist + hweight[i, j-1]
            if tmp < result[i, j-1]
                result[i, j-1] = tmp
                pq[c2i((i, j-1), shape)] = tmp
            end
        end
        if (j!=n) && (!done[i, j+1])
            tmp = ij_dist + hweight[i, j]
            if tmp < result[i, j+1]
                result[i, j+1] = tmp
                pq[c2i((i, j+1), shape)]= tmp
            end
        end
        
        if isempty(pq)
            break
        end
        
        i, j = i2c(dequeue!(pq), shape)
        ij_dist = result[i,j]
    end
    return result
end


function shortest_path_sim()

TMI_sim (generic function with 1 method)

In [23]:
n_step = 30
step_size = 10
L = 512
a = 4
d_list = 2:2:div(L, 2)-2
n_avg = 1

Profile.init(n=10^8)
Profile.clear()
@profile MI_sim(n_step, step_size, L, a, d_list, n_avg);

In [24]:
Profile.print(mincount=1000)

Overhead ╎ [+additional indent] Count File:Line; Function
    ╎83299 @Base\task.jl:356; (::IJulia.var"#15#18")()
    ╎ 83299 ...ia\src\eventloop.jl:8; eventloop(::ZMQ.Socket)
    ╎  83299 @Base\essentials.jl:709; invokelatest
    ╎   83299 @Base\essentials.jl:710; #invokelatest#1
    ╎    83299 ...execute_request.jl:67; execute_request(::ZMQ.Socket...
    ╎     83299 ...oftGlobalScope.jl:65; softscope_include_string(::M...
  81╎    ╎ 83299 @Base\loading.jl:1091; include_string(::Function,...
    ╎    ╎  83172 In[21]:105; MI_sim(::Int64, ::Int64, :...
    ╎    ╎   82499 In[21]:88; cal_MI(::Int64, ::Int64, :...
    ╎    ╎    1012  In[21]:27; my_dijkstra(::Array{Float...
 337╎    ╎    1819  In[21]:38; my_dijkstra(::Array{Float...
    ╎    ╎     1425  @Base\operators.jl:202; !=
1425╎    ╎    ╎ 1425  @Base\promotion.jl:398; ==
  67╎    ╎    2052  In[21]:40; my_dijkstra(::Array{Float...
1724╎    ╎     1724  @Base\array.jl:810; getindex
  79╎    ╎    10303 In[21]:42; my_dijkstra(::Array{Float